<a href="https://colab.research.google.com/github/sai2119/Tensorflow2.0/blob/master/MNIST_USING_DEEP_LEARNING_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING LIBRARIES

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

# Loading data

In [0]:
import tensorflow_datasets as tfds
mnist_data, mnist_info = tfds.load("mnist", with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_data["train"], mnist_data["test"]
assert isinstance(mnist_train, tf.data.Dataset)

# spliting train data with 10 % to get validation data
num_validation_sample =0.1*mnist_info.splits['train'].num_examples
#coverting validation to int 64
num_validation_sample=tf.cast(num_validation_sample,tf.int64)
# same to test data
num_test_sample = mnist_info.splits['test'].num_examples
#coverting test to int 64
num_test_sample=tf.cast(num_test_sample,tf.int64)



# custm fuction to scale 0 to 1 
def scale(image, label):
  # cast to make sure that all date in float 
  image =tf.cast(image, tf.float32)
  # As we KNow grey scale is 0 to 255 we divide the iamge by 255 to get o to 1
  image/=255.
  return image,label

scaled_train_and_validation_data =mnist_train.map(scale)
# dataset.map(*function*)applies a customtransformation to agiven dataset it take as input a funxtion which determines the transformation
 # scaling test value
scaled_test_data =mnist_test.map(scale)

In [0]:
#shuffled data 
# helps to diorder the formats of data with same information
BUFFER_SIZE =10000
# buffer size is used when data is to high it says tensor flow to perform buffer size shuffles 
#buffer_size =1 no shuffing will actually happen
# If buffer_size >=num_samples,shuffling will happen at once (uniformly)
# If 1<Buffer_size<num_samples, we will be optimizing the computational power

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_sample)
train_data =shuffled_train_and_validation_data.skip(num_validation_sample)


In [0]:
# batching
# batchsize =1=sgd
# batch size = # samples =(single batch) GD
BATCH_SIZE =150
# method dataset.batch(batch_size)
# a method that combines the consecutive elements of a dataset into batches

train_data = train_data.batch(BATCH_SIZE)
# no need for validation because we only use feed forward for validation we just calculate loss
# but model excepts validation to be in batchs 
validation_data = validation_data.batch(num_validation_sample)
test_data =scaled_test_data.batch(num_test_sample)

# iter() creates an object which can be iterated one element at atime (e.g in afor loop or while loop)
validation_inputs, validation_targets = next(iter(validation_data))


# Model

In [0]:
# image is in the size of 28*28 so input size is 784
input_size =784

# output is selected 10 because of ten layers 
output_size = 10
# this hyper parameter than can be shuffled 
hidden_layer_size =500
#the underlying assumption is that all hidden layers are of same size  

In [0]:
model = tf.keras.Sequential([
                            # our data(from tfds) is such that each input is 28*28*1 
                            # tf.keras.layers.Flatten(original shape) transforms ((flattens)a tensor into vector)
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            # activation function relu, sigmoid (logistic), tanh(prabolic function), softmax
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                            tf.keras.layers.Dense(output_size, activation='softmax')

                            ])

optimizer and the loss fuction

In [0]:
#optimizer are 3 in tf2 ada,rem,ada+movmentum (adam) best is adam 
# Loss function as it is classification problem we cross entropy in tf we have 3 types
# binary , categorical_crossentropy, sparse_categorical_crossentropy(one-hot encoding)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Training

In [8]:
# At the beginning of each epoch, the training loss will be set to 0
# the algorithm will iterate over a preset number of batches,all from train_data
# the weights and bias will be updated as many times as there are batches
# we will get a value for the loss function indicating how the training is going
# we will also see a training accuracy
# at the end of the epoch the alogorithm will forward propagate the whole validation set
# when we reach the maximum number of epochs the training will be over
NUM_EPOCHS = 10
model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs,validation_targets), verbose=2)


Epoch 1/10
360/360 - 25s - loss: 0.2692 - accuracy: 0.9182 - val_loss: 0.1120 - val_accuracy: 0.9655
Epoch 2/10
360/360 - 25s - loss: 0.1236 - accuracy: 0.9653 - val_loss: 0.0959 - val_accuracy: 0.9735
Epoch 3/10
360/360 - 26s - loss: 0.0931 - accuracy: 0.9738 - val_loss: 0.0802 - val_accuracy: 0.9778
Epoch 4/10
360/360 - 25s - loss: 0.0793 - accuracy: 0.9781 - val_loss: 0.0765 - val_accuracy: 0.9790
Epoch 5/10
360/360 - 25s - loss: 0.0665 - accuracy: 0.9818 - val_loss: 0.0544 - val_accuracy: 0.9857
Epoch 6/10
360/360 - 25s - loss: 0.0560 - accuracy: 0.9851 - val_loss: 0.0746 - val_accuracy: 0.9813
Epoch 7/10
360/360 - 26s - loss: 0.0523 - accuracy: 0.9858 - val_loss: 0.0498 - val_accuracy: 0.9877
Epoch 8/10
360/360 - 25s - loss: 0.0509 - accuracy: 0.9861 - val_loss: 0.0431 - val_accuracy: 0.9900
Epoch 9/10
360/360 - 25s - loss: 0.0425 - accuracy: 0.9892 - val_loss: 0.0438 - val_accuracy: 0.9892
Epoch 10/10
360/360 - 25s - loss: 0.0381 - accuracy: 0.9898 - val_loss: 0.0593 - val_accura

evaluate

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.1080 - accuracy: 0.9752


In [10]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.11. Test accuracy: 97.52%


# Adjustments

There are several main adjustments you may try.

Please pay attention to the time it takes for each epoch to conclude.

Using the code from the lecture as the basis, fiddle with the hyperparameters of the algorithm.



Base model



*   batch size 100
*   hidden layer 50
*   number of hidden layer = 2
*   activation function = relu for hidden layer 
*   activation function = soft max for output layer 
*   excution time = 11 sec
*   loss function(trian, validation) =(0.0983,0.0939)
*   accuracy(trian, validation) =
*   loss function(test) =0.11
*   accuracy(test) =96.94%

1. The *width* (the hidden layer size) of the algorithm. Try a hidden layer size of 200. How does the validation accuracy of the model change? What about the time it took the algorithm to train? Can you find a hidden layer size that does better?



Base model



*   batch size 200
*   hidden layer 50
*   number of hidden layer = 2
*   activation function = relu for hidden layer 
*   activation function = soft max for output layer 
*   excution time = 11 sec
*   loss function(trian, validation) =(0.1188,0.1113)
*   accuracy(trian, validation) =(0.9655,0.9673)
*   loss function(test) =0.13
*   accuracy(test) = 96.11%


2. The *depth* of the algorithm. Add another hidden layer to the algorithm. This is an extremely important exercise! How does the validation accuracy change? What about the time it took the algorithm to train? Hint: Be careful with the shapes of the weights and the biases.



Base model



*   batch size 100
*   hidden layer 50
*   number of hidden layer = 3
*   activation function = relu for hidden layer 
*   activation function = soft max for output layer 
*   excution time = 10s
*   loss function(trian, validation) =0.1068,0.1068
*   accuracy(trian, validation) =0.9680,0.9687
*   loss function(test) =0.12
*   accuracy(test) =96.28%



3. The *width and depth* of the algorithm. Add as many additional layers as you need to reach 5 hidden layers. Moreover, adjust the width of the algorithm as you find suitable. How does the validation accuracy change? What about the time it took the algorithm to train?



Base model



*   batch size 100
*   hidden layer 50
*   number of hidden layer = 5
*   activation function = relu for hidden layer 
*   activation function = soft max for output layer 
*   excution time = 11s
*   loss function(trian, validation) =0.1062,0.1067
*   accuracy(trian, validation) =0.9678, 0.9663
*   loss function(test) =0.12.
*   accuracy(test) =96.35%


# 4. Fiddle with the activation functions. Try applying sigmoid transformation to both layers. The sigmoid activation is given by the string 'sigmoid'.



Base model



*   batch size 100
*   hidden layer 50
*   number of hidden layer = 5
*   activation function = sigmoid for hidden layer 
*   activation function = sigmoid for output layer 
*   excution time = 11s
*   loss function(trian, validation) =0.3883,0.3469
*   accuracy(trian, validation) =0.9078,0.9193
*   loss function(test) =0.36. 

other model
*   batch size 100
*   hidden layer 50
*   number of hidden layer = 5
*   activation function = softmax for hidden layer 
*   activation function = softmax for output layer 
*   excution time = 11s
*   loss function(trian, validation) =2.3012, 2.3013
*   accuracy(trian, validation) =0.1129, 0.1090
*   loss function(test) =2.30. 
*   Test accuracy: 11.35%

other model
*   batch size 100
*   hidden layer 50
*   number of hidden layer = 5
*   activation function = sigmoid for hidden layer 
*   activation function = softmax for output layer 
*   excution time = 11s
*   loss function(trian, validation) =0.4351,0.3854
*   accuracy(trian, validation) =0.9093,0.9095
*   loss function(test) = 0.40.
*   Test accuracy:0.9095

other model
*   batch size 100
*   hidden layer 50
*   number of hidden layer = 5
*   activation function = relu for hidden layer 
*   activation function = relu for output layer 
*   excution time = 11s
*   loss function(trian, validation) =2.2964, 2.2984
*   accuracy(trian, validation) =0.1002,0.0990
*   loss function(test) =2.30
*   Test accuracy:9.92%

other model
*   batch size 100
*   hidden layer 50
*   number of hidden layer = 5
*   activation function = tanh for hidden layer 
*   activation function = tanh for output layer 
*   excution time = 10s
*   loss function(trian, validation) =2.1913 2.1900 
*   accuracy(trian, validation) =0.2215 0.2220
*   loss function(test) = 2.19.
*   Test accuracy: 22.34%

other model
*   batch size 100
*   hidden layer 50
*   number of hidden layer = 5
*   activation function = tanh for hidden layer 
*   activation function = softmax for output layer 
*   excution time = 10s
*   loss function(trian, validation) = 0.0974 0.0930  
*   accuracy(trian, validation) = 0.9708 0.9738
*   loss function(test) = 0.12.
*   Test accuracy:  96.36%


# 5. Fiddle with the activation functions. Try applying a ReLu to the first hidden layer and tanh to the second one. The tanh activation is given by the string 'tanh'.



Base model



*   batch size 100
*   hidden layer 50
*   number of hidden layer = 4
*   activation function = relu for 1,2 hidden layer tanh 3,4 layer 
*   activation function = soft max for output layer 
*   excution time = 11 s
*   loss function(trian, validation) =0.0284 0.0363
*   accuracy(trian, validation) =0.9913 0.9885
*   loss function(test) =0.10
*   accuracy(test) = 97.34%


# 6. Adjust the batch size. Try a batch size of 10000. How does the required time change? What about the accuracy?



Base model


*   batch size 10000
*   hidden layer 50
*   number of hidden layer = 4
*   activation function = relu for 1,2 hidden layer tanh 3,4 layer 
*   activation function = soft max for output layer 
*   excution time = 8s 
*   loss function(trian, validation) =0.1503 0.1436
*   accuracy(trian, validation) = 0.9551  0.9583
*   loss function(test) = 0.15
*   accuracy(test) =95.44%
 

# 7. Adjust the batch size. Try a batch size of 1. That's the SGD. How do the time and accuracy change? Is the result coherent with the theory?



Base model



*   batch size 1
*   hidden layer 50
*   number of hidden layer = 2
*   activation function = relu for hidden layer 
*   activation function = soft max for output layer 
*   excution time = 84s 
*   loss function(trian, validation) =0.1251 0.1158
*   accuracy(trian, validation) = 0.9672  0.9687
*   loss function(test) = 0.12.
*   accuracy(test) = 96.78%


# 8. Adjust the learning rate. Try a value of 0.0001. Does it make a difference?



Base model



*   batch size 100
*   hidden layer 50
*   number of hidden layer = 4
*   activation function = relu for1,2 hidden layer 3,4 tanh hidden layer 
*   activation function = soft max for output layer 
*   excution time = 11s 
*   loss function(trian, validation) =0.1574 0.1546 
*   accuracy(trian, validation) = 0.9555  0.9552
*   loss function(test) =0.16.
*   accuracy(test) =95.25%


# 9. Adjust the learning rate. Try a value of 0.02. Does it make a difference?



Base model



*   batch size 100
*   hidden layer 50
*   number of hidden layer = 4
*   activation function = relu for1,2 hidden layer 3,4 tanh hidden layer 
*   activation function = soft max for output layer 
*   excution time = 11s 
*   loss function(trian, validation) =0.18574 0.15846 
*   accuracy(trian, validation) = 0.8555  0.8552
*   loss function(test) =0.196.
*   accuracy(test) =80.25%